In [123]:
import numpy as np
import re
import itertools
import collections
import keras

from keras.models import Sequential
from keras.layers import Dense, Activation

In [91]:
M = np.load('word2vec.npy')
word2index = dict()
with open('word_lst.txt') as file:
    for counter, line in enumerate(file):
        word = line.strip()
        word2index[word] = counter

def word2vec(word):
    idx = word2index.get(word)
    if idx is None:
        return np.zeros(200)
    return M[idx]

        
def get_vec(word):
    try:
        retval = M[word2index[word]]
    except KeyError:
        retval = np.zeros(200)

In [5]:
def import_dataset(filename):
    
    start_tag = '<S>'
    end_tag = '</S>'
    
    sentence_lst = [] # Sentence is a list word which is list of candidate roots
    sentence_correct_lst = []
    with open(filename) as file:
        start_parsing = False
        for line in file:
            if start_parsing:
                if line.startswith(end_tag):
                    start_parsing = False
                    sentence_lst.append(sentence)
                    sentence_correct_lst.append(sentence_correct)
                else:
                    root_set = set()
                    candidate_lst = line.split()[1:]
                    for parse in candidate_lst:
                        try:
                            root_candidate = parse[:parse.index('+')]
                        except ValueError:
                            continue
                        root_set.add(root_candidate.lower())
                    if root_set:
                        sentence.append(list(root_set))
                    
                        correct = candidate_lst[0][:candidate_lst[0].index('+')]
                        sentence_correct.append(correct.lower())
                        
                
            else:
                if line.startswith(start_tag):
                    start_parsing = True
                    sentence = []
                    sentence_correct = []
                    
        return sentence_correct_lst, sentence_lst

In [7]:
train_dataset_filename = 'dataset/train.merge'
train_sentence_correct_lst, train_sentence_lst = import_dataset(train_dataset_filename)

In [17]:
train_sentence_lst

[[['döviz'], ['asker', 'askerlik'], ['20'], ['bin'], ['mark']],
 [['milli', 'mil'],
  ['savun'],
  ['bakan', 'bakanlık'],
  ['döviz'],
  ['asker', 'askerlik'],
  ['yarar'],
  ['için', 'iç'],
  ['yurt'],
  ['dış', 'dışı'],
  ['bulun', 'bul'],
  ['süre', 'sür'],
  ['3'],
  ['yıl'],
  ['1'],
  ['yıl'],
  ['in'],
  [','],
  ['döviz'],
  ['asker', 'askerlik'],
  ['rakam'],
  ['iki'],
  ['kat', 'katı'],
  ['çık'],
  ['planla']],
 [['evren', 'evre'], ['değer', 'değ'], ['-'], ['ankara']],
 [['milli', 'mil'],
  ['savun'],
  ['bakan', 'bakanlık'],
  ['döviz'],
  ['asker', 'askerlik'],
  ['konu'],
  ['çöz', 'çözüm'],
  ['ara'],
  ['girdi', 'gir']],
 [['bakan', 'bakanlık'],
  ['birçok'],
  ['kişi'],
  [','],
  ['bulun', 'bul'],
  ['ve'],
  ['çalış'],
  ['ülke'],
  ['vatandaşlık', 'vatandaş'],
  ['geç'],
  ['türkiye'],
  ['asker', 'askerlik'],
  ['yüküm'],
  ['ye', 'yer', 'yerin'],
  ['getir'],
  ['üz', 'üzer'],
  ['alternatif'],
  ['üz', 'üzer'],
  ['çalış'],
  ['başla']],
 [['türk'],
  ['vatandaş

In [109]:

model = Sequential([
    Dense(800, input_shape=(400,)),
    Activation('relu'),
    Dense(40),
    Activation('relu'),
    Dense(2),
    Activation('softmax'),
])
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

def generate_samples(sentences):
    for sentence in sentences:
        for w1, w2 in zip(sentence, sentence[1:]):
            itert = itertools.product(w1,w2)
            yield next(itert), 1
            for others in itert:
                yield others, 0
samples = list(generate_samples(train_sentence_lst))

In [25]:
samples[:10]

[(('döviz', 'asker'), 1),
 (('döviz', 'askerlik'), 0),
 (('asker', '20'), 1),
 (('askerlik', '20'), 0),
 (('20', 'bin'), 1),
 (('bin', 'mark'), 1),
 (('milli', 'savun'), 1),
 (('mil', 'savun'), 0),
 (('savun', 'bakan'), 1),
 (('savun', 'bakanlık'), 0)]

In [110]:
def gen_subsamples():
    subsample_size = 10000
    for subsample in range(0, len(samples), subsample_size):
        subsamples = samples[subsample: subsample + subsample_size]
        train_data = np.array([np.append(word2vec(w1), word2vec(w2)) for (w1,w2),_ in subsamples])
        train_labels = np.array([[v==0, v==1] for _, v in subsamples])
        print(subsample, len(samples))
        yield train_data, train_labels
#train_data, train_labels = next(gen_subsamples())
#model.fit(train_data, train_labels, epochs=10, batch_size=32)
for train_data, train_labels in gen_subsamples():
    model.fit(train_data, train_labels, epochs=10, batch_size=32)

0 1164279
Epoch 1/10
10000/10000 [==============================] - 1s 148us/step - loss: 0.5944 - acc: 0.7121
Epoch 2/10
10000/10000 [==============================] - 1s 118us/step - loss: 0.4872 - acc: 0.7696
Epoch 3/10
10000/10000 [==============================] - 1s 120us/step - loss: 0.4207 - acc: 0.8051
Epoch 4/10
10000/10000 [==============================] - 1s 120us/step - loss: 0.3685 - acc: 0.8356
Epoch 5/10
10000/10000 [==============================] - 1s 119us/step - loss: 0.3301 - acc: 0.8523
Epoch 6/10
10000/10000 [==============================] - 1s 119us/step - loss: 0.3039 - acc: 0.8673
Epoch 7/10
10000/10000 [==============================] - 1s 122us/step - loss: 0.2905 - acc: 0.8759
Epoch 8/10
10000/10000 [==============================] - 1s 128us/step - loss: 0.2737 - acc: 0.8810
Epoch 9/10
10000/10000 [==============================] - 1s 120us/step - loss: 0.2622 - acc: 0.8864
Epoch 10/10
10000/10000 [==============================] - 1s 121us/step - loss: 

80000 1164279
Epoch 1/10
10000/10000 [==============================] - 1s 135us/step - loss: 0.5820 - acc: 0.7711
Epoch 2/10
10000/10000 [==============================] - 1s 136us/step - loss: 0.4587 - acc: 0.8081
Epoch 3/10
10000/10000 [==============================] - 1s 134us/step - loss: 0.4060 - acc: 0.8310
Epoch 4/10
10000/10000 [==============================] - 1s 139us/step - loss: 0.3650 - acc: 0.8474
Epoch 5/10
10000/10000 [==============================] - 1s 142us/step - loss: 0.3391 - acc: 0.8584
Epoch 6/10
10000/10000 [==============================] - 1s 141us/step - loss: 0.3216 - acc: 0.8682
Epoch 7/10
10000/10000 [==============================] - 1s 138us/step - loss: 0.3049 - acc: 0.8763
Epoch 8/10
10000/10000 [==============================] - 1s 144us/step - loss: 0.2922 - acc: 0.8795
Epoch 9/10
10000/10000 [==============================] - 1s 144us/step - loss: 0.2877 - acc: 0.8831
Epoch 10/10
10000/10000 [==============================] - 1s 145us/step - lo

10000/10000 [==============================] - 1s 139us/step - loss: 0.3153 - acc: 0.8736
160000 1164279
Epoch 1/10
10000/10000 [==============================] - 1s 136us/step - loss: 0.5714 - acc: 0.7681
Epoch 2/10
10000/10000 [==============================] - 1s 136us/step - loss: 0.4709 - acc: 0.8044
Epoch 3/10
10000/10000 [==============================] - 1s 137us/step - loss: 0.4246 - acc: 0.8263
Epoch 4/10
10000/10000 [==============================] - 1s 137us/step - loss: 0.3898 - acc: 0.8390
Epoch 5/10
10000/10000 [==============================] - 1s 136us/step - loss: 0.3676 - acc: 0.8496
Epoch 6/10
10000/10000 [==============================] - 1s 137us/step - loss: 0.3477 - acc: 0.8573
Epoch 7/10
10000/10000 [==============================] - 1s 135us/step - loss: 0.3321 - acc: 0.8641
Epoch 8/10
10000/10000 [==============================] - 1s 138us/step - loss: 0.3235 - acc: 0.8636
Epoch 9/10
10000/10000 [==============================] - 1s 144us/step - loss: 0.3181 

Epoch 10/10
10000/10000 [==============================] - 1s 135us/step - loss: 0.3201 - acc: 0.8497
240000 1164279
Epoch 1/10
10000/10000 [==============================] - 1s 136us/step - loss: 0.6497 - acc: 0.7716
Epoch 2/10
10000/10000 [==============================] - 1s 139us/step - loss: 0.5148 - acc: 0.7978
Epoch 3/10
10000/10000 [==============================] - 1s 140us/step - loss: 0.4663 - acc: 0.8131
Epoch 4/10
10000/10000 [==============================] - 1s 143us/step - loss: 0.4276 - acc: 0.8240
Epoch 5/10
10000/10000 [==============================] - 1s 139us/step - loss: 0.3942 - acc: 0.8349
Epoch 6/10
10000/10000 [==============================] - 1s 135us/step - loss: 0.3761 - acc: 0.8408
Epoch 7/10
10000/10000 [==============================] - 1s 135us/step - loss: 0.3610 - acc: 0.8449
Epoch 8/10
10000/10000 [==============================] - 1s 137us/step - loss: 0.3519 - acc: 0.8482
Epoch 9/10
10000/10000 [==============================] - 1s 135us/step - l

10000/10000 [==============================] - 1s 134us/step - loss: 0.3467 - acc: 0.8486
Epoch 10/10
10000/10000 [==============================] - 1s 135us/step - loss: 0.3391 - acc: 0.8502
320000 1164279
Epoch 1/10
10000/10000 [==============================] - 1s 136us/step - loss: 0.5859 - acc: 0.7908
Epoch 2/10
10000/10000 [==============================] - 1s 136us/step - loss: 0.4836 - acc: 0.8149
Epoch 3/10
10000/10000 [==============================] - 1s 136us/step - loss: 0.4346 - acc: 0.8321
Epoch 4/10
10000/10000 [==============================] - 1s 137us/step - loss: 0.3924 - acc: 0.8380
Epoch 5/10
10000/10000 [==============================] - 1s 136us/step - loss: 0.3650 - acc: 0.8466
Epoch 6/10
10000/10000 [==============================] - 1s 138us/step - loss: 0.3495 - acc: 0.8505
Epoch 7/10
10000/10000 [==============================] - 1s 137us/step - loss: 0.3334 - acc: 0.8528
Epoch 8/10
10000/10000 [==============================] - 1s 141us/step - loss: 0.3287

...

10000/10000 [==============================] - 1s 134us/step - loss: 0.4454 - acc: 0.8200
Epoch 7/10
10000/10000 [==============================] - 2s 152us/step - loss: 0.4269 - acc: 0.8248
Epoch 8/10
10000/10000 [==============================] - 1s 137us/step - loss: 0.4234 - acc: 0.8261
Epoch 9/10
10000/10000 [==============================] - 1s 131us/step - loss: 0.4146 - acc: 0.8297
Epoch 10/10
10000/10000 [==============================] - 1s 132us/step - loss: 0.4165 - acc: 0.8316
800000 1164279
Epoch 1/10
10000/10000 [==============================] - 1s 137us/step - loss: 0.5635 - acc: 0.8032
Epoch 2/10
10000/10000 [==============================] - 1s 149us/step - loss: 0.4997 - acc: 0.8145
Epoch 3/10
10000/10000 [==============================] - 1s 134us/step - loss: 0.4692 - acc: 0.8268
Epoch 4/10
10000/10000 [==============================] - 1s 137us/step - loss: 0.4487 - acc: 0.8322
Epoch 5/10
10000/10000 [==============================] - 1s 137us/step - loss: 0.4328

Epoch 6/10
10000/10000 [==============================] - 1s 137us/step - loss: 0.4825 - acc: 0.8149
Epoch 7/10
10000/10000 [==============================] - 1s 145us/step - loss: 0.4743 - acc: 0.8152
Epoch 8/10
10000/10000 [==============================] - 2s 172us/step - loss: 0.4734 - acc: 0.8180
Epoch 9/10
10000/10000 [==============================] - 2s 156us/step - loss: 0.4748 - acc: 0.8171
Epoch 10/10
10000/10000 [==============================] - 2s 160us/step - loss: 0.4608 - acc: 0.8187
880000 1164279
Epoch 1/10
10000/10000 [==============================] - 1s 139us/step - loss: 0.6178 - acc: 0.7862
Epoch 2/10
10000/10000 [==============================] - 1s 142us/step - loss: 0.5480 - acc: 0.7994
Epoch 3/10
10000/10000 [==============================] - 1s 143us/step - loss: 0.5386 - acc: 0.8094
Epoch 4/10
10000/10000 [==============================] - 1s 145us/step - loss: 0.5059 - acc: 0.8125
Epoch 5/10
10000/10000 [==============================] - 1s 145us/step - l

10000/10000 [==============================] - 1s 136us/step - loss: 0.4926 - acc: 0.8115
Epoch 6/10
10000/10000 [==============================] - 1s 142us/step - loss: 0.4896 - acc: 0.8151
Epoch 7/10
10000/10000 [==============================] - 1s 140us/step - loss: 0.4735 - acc: 0.8168
Epoch 8/10
10000/10000 [==============================] - 1s 143us/step - loss: 0.4724 - acc: 0.8192
Epoch 9/10
10000/10000 [==============================] - 1s 135us/step - loss: 0.4690 - acc: 0.8196
Epoch 10/10
10000/10000 [==============================] - 1s 142us/step - loss: 0.4565 - acc: 0.8210
960000 1164279
Epoch 1/10
10000/10000 [==============================] - 2s 152us/step - loss: 0.6009 - acc: 0.7823
Epoch 2/10
10000/10000 [==============================] - 2s 153us/step - loss: 0.5622 - acc: 0.7927
Epoch 3/10
10000/10000 [==============================] - 1s 136us/step - loss: 0.5286 - acc: 0.7981
Epoch 4/10
10000/10000 [==============================] - 1s 135us/step - loss: 0.5149

10000/10000 [==============================] - 1s 131us/step - loss: 0.5846 - acc: 0.7930
Epoch 5/10
10000/10000 [==============================] - 1s 130us/step - loss: 0.5808 - acc: 0.7957
Epoch 6/10
10000/10000 [==============================] - 1s 135us/step - loss: 0.5531 - acc: 0.7980
Epoch 7/10
10000/10000 [==============================] - 1s 136us/step - loss: 0.5506 - acc: 0.8003
Epoch 8/10
10000/10000 [==============================] - 1s 135us/step - loss: 0.5459 - acc: 0.8020
Epoch 9/10
10000/10000 [==============================] - 1s 130us/step - loss: 0.5311 - acc: 0.8038
Epoch 10/10
10000/10000 [==============================] - 1s 132us/step - loss: 0.5374 - acc: 0.8049
1040000 1164279
Epoch 1/10
10000/10000 [==============================] - 1s 143us/step - loss: 0.6863 - acc: 0.7525
Epoch 2/10
10000/10000 [==============================] - 1s 134us/step - loss: 0.6279 - acc: 0.7589
Epoch 3/10
10000/10000 [==============================] - 1s 135us/step - loss: 0.590

10000/10000 [==============================] - 1s 139us/step - loss: 0.5630 - acc: 0.7716
Epoch 4/10
10000/10000 [==============================] - 1s 138us/step - loss: 0.5470 - acc: 0.7788
Epoch 5/10
10000/10000 [==============================] - 1s 140us/step - loss: 0.5376 - acc: 0.7807
Epoch 6/10
10000/10000 [==============================] - 1s 131us/step - loss: 0.5379 - acc: 0.7828
Epoch 7/10
10000/10000 [==============================] - 1s 139us/step - loss: 0.5198 - acc: 0.7848
Epoch 8/10
10000/10000 [==============================] - 1s 140us/step - loss: 0.5244 - acc: 0.7856
Epoch 9/10
10000/10000 [==============================] - 1s 133us/step - loss: 0.5250 - acc: 0.7877
Epoch 10/10
10000/10000 [==============================] - 1s 134us/step - loss: 0.5138 - acc: 0.7884
1120000 1164279
Epoch 1/10
10000/10000 [==============================] - 1s 135us/step - loss: 0.6976 - acc: 0.7599
Epoch 2/10
10000/10000 [==============================] - 1s 134us/step - loss: 0.633

In [111]:
model.save("dense_800_40_2.keras")

In [124]:
model = keras.models.load_model("dense_800_40_2.keras")

In [125]:
model.predict(np.array([np.append(word2vec("asd"), word2vec("asd"))]))

array([[0.27538309, 0.72461694]], dtype=float32)

In [85]:
dataset_filename = 'dataset/test.merge'
sentence_correct_lst, sentence_lst = import_dataset(dataset_filename)

In [116]:
class ScoreModel:
    def __init__(self, verbose=False):
        self.verbose = verbose
    
    def predict(self, sentence):
        max_score = float('-inf')
        predict_sentence = None
        self._cache = {}
        for element in itertools.product(*sentence):
            score = self.calc_sentence_score(element)
            if self.verbose:
                print(element)
                print('Score: %.2f' % score)
                print()
            if score > max_score:
                max_score = score
                predict_sentence = list(element)
        return predict_sentence
    
    def pair_score(self, word1, word2):
        if (word1, word2) in self._cache:
            return self._cache[(word1, word2)]
        try:
            vec1 = M[word2index[word1]]
            vec2 = M[word2index[word2]]
        except:
            return 0
        #return np.abs(np.dot(vec1, vec2))
        self._cache[(word1, word2)] = model.predict(np.array([np.append(vec1, vec2)]))[0][1]
        return self._cache[(word1, word2)]

    def calc_sentence_score(self, sentence):
        score = 0

        if len(sentence) <= 1:
            return score

        for i in range(len(sentence) - 1):
            word1 = sentence[i]
            word2 = sentence[i + 1]
            score += self.pair_score(word1, word2)

        return score

In [117]:
correct_count = 0
false_count = 0

mdl = ScoreModel()
for num, (sentence, sentence_correct) in enumerate(zip(sentence_lst, sentence_correct_lst)):
    predict_sentence = mdl.predict(sentence)
    for word1, word2 in zip(predict_sentence, sentence_correct):
        if word1 == word2:
            correct_count += 1
        else:
            false_count += 1

total_count = correct_count + false_count
accuracy = correct_count / total_count

print('Total number of words : %s' % total_count)
print('Correctly predicted : %s' % correct_count)
print('Accuracy : %.3f' % accuracy)

Total number of words : 861
Correctly predicted : 809
Accuracy : 0.940


In [118]:
def parse_sentence(sentence):
    sentence = '\'' + sentence + '\''
    word_lst = !./trnltk/parser.py {sentence}
    #print(word_lst)
    retval = []
    for word in word_lst:
        root_lst = word.split()
        retval.append(root_lst)
    return retval

    yüklenen yükle+Verb^DB+Verb+Pass+Pos^DB+Adj+PresPart yük+Noun+A3sg+Pnon+Nom^DB+Verb+Acquire+Pos^DB+Adj+PresPart

    Sentence Examples
    
    dolar fiyatları beş TL seviyesinde bulunurken Euro fiyatları altı TL seviyesinde hareket ediyor
    yorulunca alın damarları gözükmeye başladı
    istediğiniz kadar ürün alın 
    sözlerine çok alındı (Wrong)
    ya iyi olarak ölürsün ya da kötüye dönüşecek kadar uzun yaşarsın
    nedir amacımız bunu göndermekle uzaylılara karsı bir sinerji yaratalım dostluk olsun mu
    kafanızı kullansaydınız o taşların doğada bulunan 4 elementi simgelediğini anlardınız

In [119]:
sentence = 'istediğiniz kadar ürün alın'
parsed_sentence = parse_sentence(sentence)

combination_count = 1
for word in parsed_sentence:
  combination_count *= len(word)
print('Total number of possibilities : %s' % combination_count)

print(parsed_sentence)
print(sentence)

vmodel = ScoreModel(verbose=True)
prediction = vmodel.predict(parsed_sentence)

print('-------')
print('Predicted roots : ')
print(prediction)

Total number of possibilities : 7
[['/bin/bash:', './trnltk/parser.py:', 'No', 'such', 'file', 'or', 'directory']]
istediğiniz kadar ürün alın
('/bin/bash:',)
Score: 0.00

('./trnltk/parser.py:',)
Score: 0.00

('No',)
Score: 0.00

('such',)
Score: 0.00

('file',)
Score: 0.00

('or',)
Score: 0.00

('directory',)
Score: 0.00

-------
Predicted roots : 
['/bin/bash:']
